In [1]:
import numpy as np
import mne
from mne.preprocessing import read_ica
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.svm import SVC, SVR
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from mne.decoding import CSP, SPoC
from continuous_control_bci.util import emg_classes_to_eeg_classes, SUBJECT_IDS
from tqdm import tqdm
import matplotlib as plt

from sklearn.pipeline import FeatureUnion
from continuous_control_bci.data.load_data import load_calibration, load_driving
from continuous_control_bci.data.preprocessing import make_epochs, epochs_to_train_test
from continuous_control_bci.modelling.csp_classifier import create_csp_classifier, get_driving_epochs_for_csp
from scipy.signal import cheby2, sosfiltfilt
from sklearn.metrics import f1_score

In [2]:
# Subject 61 should have fairly good EEG data and performance
subject_id = "061"
include_rest = True
raw = load_calibration(subject_id)
raw.set_eeg_reference()
raw.filter(l_freq=5, h_freq=35)

ica = read_ica(f'D:/RUG/Term 2/FYRP/data/ica/P{subject_id}-calibration-ica.fif')
ica.apply(raw)

epochs = make_epochs(raw, include_rest=include_rest)
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)

X_train, _, y_train, _ = epochs_to_train_test(epochs)
print("Training classifier. This may take a while..")
rank = {
    'eeg': X_train.shape[1] - len(ica.exclude),
    'mag': 32,
}
clf, y_pred = create_csp_classifier(X_train, y_train, rank)
#svm_model, y_svm_pred = create_csp_svm_classifier(X_train, y_train, rank)
print("Classifier trained!")


Extracting EDF parameters from D:\RUG\Term 2\FYRP\data\ivo_data\sub-P061\motor-imagery-csp-061-acquisition-[2024.02.07-14.35.15].gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Channel 1, Channel 2, Channel 3, Channel 4, Channel 5, Channel 6, Channel 7, Channel 8, Channel 9, Channel 10, Channel 11, Channel 12, Channel 13, Channel 14, Channel 15, Channel 16, Channel 17, Channel 18, Channel 19, Channel 20, Channel 21, Channel 22, Channel 23, Channel 24, Channel 25, Channel 26, Channel 27, Channel 28, Channel 29, Channel 30, Channel 31, Channel 32, EX 1, EX 2, EX 3, EX 4, EX 5, EX 6, EX 7, EX 8
Creating raw.info structure...
Reading 0 ... 882783  =      0.000 ...   431.046 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter paramete

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading D:/RUG/Term 2/FYRP/data/ica/P061-calibration-ica.fif ...
Now restoring ICA solution ...
Ready.
Applying ICA to Raw instance


[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.4s finished


    Transforming to ICA space (32 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Used Annotations descriptions: ['769', '770']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['800']
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 40 events and 7681 original time points ...
0 bad epochs dropped
Not setting metadata
80 matching events found
No baseline correction applied
Training classifier. This may take a while..


D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\preprocessing.py:83: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([epochs, rest_epochs], add_offset=False)
D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\preprocessing.py:83: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.concatenate_epochs([epochs, rest_epochs], add_offset=False)


Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30

In [3]:
# Function for filtering the signal
freq_bands = [[4,8],[4,12],[8,12],[10,14],[12,16],[13,30]]

def filter_bank(raw:np.ndarray, freq_bands, order:int):
    filtered_signals = [[] for _ in range(len(freq_bands))]
    for i, freq_band in enumerate(freq_bands):
        iir_params = dict(order=order, ftype="cheby2", output='sos', rs=10) # what exactly is rs? How do i know which order to use?
        iir_params = mne.filter.construct_iir_filter(iir_params, f_pass=freq_band, sfreq=2048, btype="bandpass")
        filtered_raw = sosfiltfilt(iir_params['sos'], raw)
        filtered_signals[i].append(filtered_raw)
    filtered_signals = np.asarray(filtered_signals)
    filtered_signals = filtered_signals[:,0]
    return filtered_signals


x = filter_bank(epochs.get_data(picks=eeg_channel_indices),freq_bands, 4)


# Apply CSP to all filter bands
# Then SKlearns mutual_info_classif to optain best features for SVM
# Then make a pipeline for it


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 8.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 12.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 12.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 10.00, 14.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (t

In [4]:
# simply using the individual freq-ranges to make predictions
y_preds = []
clfs = []
for data in x:
    clf, y_pred = create_csp_classifier(data, y_train, rank)
    clfs.append(clf)
    y_preds.append(y_pred)

Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30

In [5]:
for pred in y_preds:
    print(classification_report(y_train, pred, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       0.44      0.40      0.42        20
       Right       0.20      0.05      0.08        20
        Rest       0.51      0.72      0.60        40

    accuracy                           0.48        80
   macro avg       0.38      0.39      0.37        80
weighted avg       0.42      0.47      0.42        80

              precision    recall  f1-score   support

        Left       0.95      0.95      0.95        20
       Right       0.81      0.85      0.83        20
        Rest       0.92      0.90      0.91        40

    accuracy                           0.90        80
   macro avg       0.89      0.90      0.90        80
weighted avg       0.90      0.90      0.90        80

              precision    recall  f1-score   support

        Left       0.94      0.85      0.89        20
       Right       0.81      0.85      0.83        20
        Rest       0.88      0.90      0.89        40

    accuracy        

# Now getting the CSP features from all bands and then finding the most informative ones

In [6]:
csp = CSP(n_components=6, reg='shrinkage', log=True, rank=rank, transform_into='average_power')
def filter_bank_csp(raw:np.ndarray, y_train, freq_bands, order:int):
    csp_channels = []
    csp_info = []
    for i, freq_band in enumerate(freq_bands):
        iir_params = dict(order=order, ftype="cheby2", output='sos', rs=10) # what exactly is rs? How do i know which order to use?
        iir_params = mne.filter.construct_iir_filter(iir_params, f_pass=freq_band, sfreq=2048, btype="bandpass")
        filtered_raw = sosfiltfilt(iir_params['sos'], raw)
        csp.fit(filtered_raw, y_train)
        tmp = csp.transform(filtered_raw)
        csp_info.append(csp)
        if i == 0:
            csp_channels = tmp
        else:
            csp_channels = np.concatenate((csp_channels, tmp), axis=1)
    return csp_channels, csp_info

In [7]:
a, csp_info = filter_bank_csp(X_train, y_train, freq_bands, 4)
a


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 8.00 Hz: -20.00, -20.00 dB

Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank={'eeg': 30, 'mag': 32}
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 32
Estimating covariance using SHRINKAGE
Done.

IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.0

array([[-0.49640754, -0.72242938, -1.05047732, ..., -0.43396845,
        -0.61082532, -0.20972365],
       [-0.64518856, -1.08417172,  0.12367374, ..., -0.46880308,
        -0.63132953, -0.443927  ],
       [-0.84039391, -0.47078607,  0.55127967, ..., -0.54662501,
        -0.86423456,  0.02998544],
       ...,
       [-0.51068572, -0.78608724, -0.40202641, ...,  0.08764144,
         1.78820574, -0.03605475],
       [-0.83349486, -0.58495126, -0.02367804, ..., -0.49518002,
         0.30145204,  0.24704696],
       [-0.26629352, -0.18727795, -0.49632512, ...,  0.20406627,
         1.57326341,  0.10517648]])

In [8]:
a.shape

(80, 36)

In [9]:
clf = LogisticRegression()
y_pred_fbcsp = cross_val_predict(clf, a, y_train, cv=5)
print(classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       1.00      0.85      0.92        20
       Right       0.90      0.95      0.93        20
        Rest       0.93      0.97      0.95        40

    accuracy                           0.94        80
   macro avg       0.94      0.92      0.93        80
weighted avg       0.94      0.94      0.94        80



# Using SelectKBest to find most informative 10 channels for prediction

In [10]:
sel_K = SelectKBest(mutual_info_classif, k=10).fit(a,y_train)
new_train = sel_K.transform(a)
new_train

array([[-1.03484160e+00,  2.77766479e-01, -1.66899207e+00,
         3.35306829e-01,  2.58645662e-01, -1.94536848e+00,
        -1.52635970e+00, -1.27797349e+00, -8.16106154e-01,
        -3.20585955e-01],
       [-1.16413969e+00, -2.71568809e-02, -1.08827299e+00,
        -3.22567050e-02, -9.22489903e-01, -2.06840071e+00,
        -1.07737891e+00, -1.91799325e+00, -6.79887145e-01,
        -1.11463354e-01],
       [-1.04570507e+00,  1.05129200e+00, -1.82142323e+00,
         1.17914301e+00,  1.04592644e+00, -1.38138875e+00,
        -1.29209656e+00, -1.41025344e+00, -6.31990360e-01,
         4.67990977e-01],
       [-4.10120974e-01, -1.26031997e+00, -2.73108348e-01,
        -1.61780768e+00, -6.73736186e-01, -1.40108371e+00,
         1.63949383e-01, -2.01697236e+00, -1.26846745e+00,
        -1.16481772e+00],
       [-1.34761044e+00,  6.43884086e-01, -2.09024807e+00,
         8.61979975e-01, -1.61120736e-01, -2.51558301e+00,
        -1.99750659e+00, -9.93440374e-01, -1.05254803e+00,
        -1.

In [20]:
clf = LogisticRegression()
y_pred_fbcsp = cross_val_predict(clf, new_train, y_train, cv=5)
clf.fit(new_train, y_train)
print(classification_report(y_train, y_pred_fbcsp, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       0.95      0.90      0.92        20
       Right       0.90      0.90      0.90        20
        Rest       0.93      0.95      0.94        40

    accuracy                           0.93        80
   macro avg       0.92      0.92      0.92        80
weighted avg       0.93      0.93      0.92        80



# Now apply the filters and predictor to the driving Data
 - 200ms time windows, no overlap

 - First Filtering, then CSP then predicting

In [12]:
driving = load_driving(subject_id)
driving.raw.set_eeg_reference()
driving.raw.filter(l_freq=5, h_freq=35)
ica.apply(driving.raw)

# EEG Data Stream
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
eeg_data, eeg_time = driving.raw[eeg_channel_indices]
# Those windows are basically my epochs on which I try to predict something, if a certain amount of windows have the same prediction (3.5 sec or smth) this can then be used to controll the car
eeg_windows = mne.make_fixed_length_epochs(driving.raw, duration=0.2, preload=True, reject_by_annotation=False)

X_driving = eeg_windows.get_data(copy=False, picks=eeg_channel_indices)

# Making predictions based on EEG windows
# -1 left, 0 rest, 1 right
y_driving = driving.emg_prediction_stream['time_series']
y_driving = emg_classes_to_eeg_classes(y_driving)
y_driving = np.squeeze(y_driving)
print(X_driving.shape)
print(y_driving[:-1].shape)

Creating RawArray with float64 data, n_channels=40, n_times=5494682
    Range : 0 ... 5494681 =      0.000 ...  2682.950 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


D:\RUG\Term 2\FYRP\continuous_control_bci\continuous_control_bci\data\load_data.py:28: RuntimeWarning: The unit for channel(s) AF3, AF4, C3, C4, CP1, CP2, CP5, CP6, Cz, F3, F4, F7, F8, FC1, FC2, FC5, FC6, Fp1, Fp2, Fz, LHEOG, LVEOG, Left-extension, Left-flexion, O1, O2, Oz, P3, P4, P7, P8, PO3, PO4, Pz, RHEOG, Right-extension, Right-flexion, T7, T8, UVEOG has changed from NA to V.
  streams.raw.set_channel_types(CHANNEL_TYPE_MAPPING)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 3381 samples (1.651 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    3.9s finished


Applying ICA to Raw instance
    Transforming to ICA space (32 components)
    Zeroing out 2 ICA components
    Projecting back using 32 PCA components
Not setting metadata
13414 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 13414 events and 410 original time points ...
0 bad epochs dropped
(13414, 32, 410)
(13414,)


In [13]:
print(X_driving.shape, y_driving.shape)

(13414, 32, 410) (13415,)


In [15]:
#driving_filtered = filter_bank_csp(X_driving, y_driving[:-1], freq_bands, 4)
driving_filtered = filter_bank(X_driving, freq_bands, 4)


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 8.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 4.00, 12.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 12.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 10.00, 14.00 Hz: -20.00, -20.00 dB


IIR filter parameters
---------------------
Chebyshev II bandpass zero-phase (t

In [16]:
#driving_filtered = sel_K.transform(driving_filtered)
driving_filtered.shape

(6, 13414, 32, 410)

In [29]:
driving_csp = []
for i in range(driving_filtered.shape[0]):
    driving_ = csp_info[i].transform(driving_filtered[i])
    if i == 0:
        driving_csp = driving_
    else:
        driving_csp = np.concatenate((driving_csp, driving_), axis=1)


(13414, 12)
(13414, 18)
(13414, 24)
(13414, 30)
(13414, 36)


array([[-1.41688882, -4.37609279, -2.93298977, ..., -0.32958378,
        -0.71580626, -0.2202402 ],
       [-2.92825547, -2.20330822, -0.95556354, ..., -1.28139552,
         0.5874292 , -1.31938844],
       [-2.22333912, -0.49398921, -3.3312014 , ..., -0.32545936,
         0.29348704,  0.46837213],
       ...,
       [-1.53125519, -4.34464196, -2.23880999, ...,  1.12210198,
         0.96720295, -0.07231587],
       [-0.99952928, -1.74691633,  0.4984999 , ...,  2.39258957,
         0.40657072, -0.12102583],
       [-1.96987686, -1.94537543,  0.96942868, ...,  1.15907897,
         0.80036494,  0.49198014]])

In [31]:
driving_csp = sel_K.transform(driving_csp)

In [18]:
def predict_against_threshold(clf, X, t):
    y_driving_pred_prob = clf.predict_proba(X)
    rest = y_driving_pred_prob[:, 2] > t
    y_driving_pred = np.array([2] * len(rest))
    y_driving_pred[~rest] = y_driving_pred_prob[~rest, :2].argmax(axis=1)
    return y_driving_pred

def predict_against_threshold_indiv(clf, X, t):
    y_driving_pred_prob = clf.predict_proba(X)
    if y_driving_pred_prob [:, 2] > t:
        y_pred = 2
    elif y_driving_pred_prob [:, 0] > y_driving_pred_prob [:, 1]:
        y_pred = 0
    else:
        y_pred = 1

    return y_pred



In [33]:
driving_y_pred = predict_against_threshold(clf, driving_csp, 0.2)

In [35]:
y_preds = []
majority_pred = []
# left, right, rest
pred_direct = [0,0,0]
for i, epoch in enumerate(X_driving):
    # Predicting each 200ms windows individually
    y_driving_pred = predict_against_threshold_indiv(clf, driving_csp[i:i+1,:], 0.2)
    pred_direct[y_driving_pred] += 1
    y_preds.append(y_driving_pred)


    # Now aggregate over 2sec time window
    if i % 10 == 0:
        majority_pred.append(pred_direct.index(max(pred_direct)))
        pred_direct = [0,0,0]


y_preds = np.asarray(y_preds)
majority_pred = np.asarray(majority_pred)

driving = load_driving(subject_id)
driving.raw.set_eeg_reference()
driving.raw.filter(l_freq=5, h_freq=35)
ica.apply(driving.raw)

In [39]:
print(classification_report(y_driving[:-1], driving_y_pred, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       0.32      0.93      0.48      4338
       Right       0.05      0.01      0.02      2923
        Rest       0.85      0.03      0.06      6153

    accuracy                           0.32     13414
   macro avg       0.41      0.32      0.18     13414
weighted avg       0.50      0.32      0.18     13414



In [49]:
print(classification_report(y_driving[:-1:10], majority_pred, target_names=["Left", "Right", "Rest"]))

              precision    recall  f1-score   support

        Left       0.33      1.00      0.50       444
       Right       0.00      0.00      0.00       290
        Rest       0.80      0.01      0.01       608

    accuracy                           0.33      1342
   macro avg       0.38      0.34      0.17      1342
weighted avg       0.47      0.33      0.17      1342



array([0, 0, 0, ..., 0, 0, 0])